In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2
import shutil

In [2]:
#get csv list
data_folder = 'C:/Power BI'
csv_files = []
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        csv_files.append(file)

In [3]:
#create new folder to isolate csv's
dataset_dir = 'datasets/' #name of folder
parent_dir = 'C:/Power BI/' #path to folder
path = os.path.join(parent_dir, dataset_dir)

try:
    os.mkdir(path)
except:
    pass

In [4]:
#move csv's to isolated directory
try:
    for f in csv_files:
        shutil.copyfile('C:/Power BI/' + f, 'C:/Power BI/datasets/' + f)
except:
    pass

In [5]:
data_path = path

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file, low_memory=False, index_col=False)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="cp437", low_memory=False, index_col=False, errors='ignore')
    print('Loading ' + file + ' into dataframe')
print('loading completed')

Loading Admission Details.csv into dataframe
Loading Appointment.csv into dataframe
Loading Authorization.csv into dataframe
Loading Billing.csv into dataframe
Loading CCF026 Multi Disciplinary Discharge.csv into dataframe
Loading Chart Users.csv into dataframe
Loading Discharge Details.csv into dataframe
Loading Discharged Chart.csv into dataframe
Loading Form MH_PHQ9.csv into dataframe
Loading Form Status.csv into dataframe
Loading FormLab 1Progress Note v2.5.csv into dataframe
Loading FormLab 247 Intake v1.4.csv into dataframe
Loading FormLab 247 Intake v1.5.csv into dataframe
Loading FormLab 820 Element of Care v1.2.csv into dataframe
Loading FormLab Admission Assessment v1.10.csv into dataframe
Loading FormLab Admission Assessment v1.11.csv into dataframe
Loading FormLab Admission Assessment v1.12.csv into dataframe
Loading FormLab Admission Assessment v1.8.csv into dataframe
Loading FormLab Admission Assessment v1.9.csv into dataframe
Loading FormLab Admission Letter v1.0.csv int

In [6]:
for k in csv_files:
    
    dataframe = df[k]
    
    clean_tbl_name = k.lower().replace(" ","_").replace("-","").replace("formlab", "fl").replace(".", "_") \
    .replace("(", "").replace(")", "").replace("bradford_county_release_of_information", "bcroi")\
    .replace("bradford_county_restricted_consent_to_release","bcrroi").replace(",", "")\
    .replace("casa-trinity,_inc", "casa").replace("discharge", "dc").replace("with", "w")\
    .replace("without", "wo").replace("_csv", "").replace("___", "_").replace("__", "_")\
    .replace("admission", "admit").replace("group_transition_form", "gtf").replace("progress", "prog")\
    .replace("assessment", "ass").replace("case_management", "cm").replace("case_manager", "cm")\
    .replace("financial_obligations", "fin_ob")\
    .replace("agreement_to_the_rules_and_regulations", "ag_rl_reg").replace("universal_face_sheet", "ufs")\
    .replace("need_and_referral", "n_and_r").replace("adult", "adlt").replace("child", "chld")\
    .replace("interview","int").replace("health_measurements_section_h8_adlt_h3_chld","health_meas")\
    .replace("casatrinity_inc", "casa").replace("counselor_do_at_eval_","").replace("baseline", "bl")\
    .replace("care_survey_for_adlts", "survey").replace("element_of_care", "eoc")\
    .replace("multi_disciplinary", "multidisp").replace("identified", "id")\
    .replace("perception_of_outcomes", "poc").replace("rhio_alert", "rhio")\
    .replace("telemedicine_rules_and_expectations", "tele_r_and_e")\
    .replace("release_of_bh_information", "roi").replace("patient", "pt")\
    .replace("authorization","authtable").replace("user", "users").replace(u"\u201c", "")
    
    tbl_name = clean_tbl_name
    
    print(k + ' changing to ' + clean_tbl_name)

    #clean column names
    dataframe.columns = [x.lower().replace(" ", "_").replace("-", "").replace("#","num").replace("?", "")\
                     .replace("=","").replace("\n","").replace("\r","").replace("\r\n","").replace("]","_")\
                     .replace("]","_").replace("[","_").replace("\\","_").replace(".","_").replace("$","")\
                     .replace("%","").replace("#","").replace("(","").replace(")","").replace("?","")\
                     .replace(",","").replace("*","").replace(":","").replace("'","").replace("&","")\
                     .replace(";","").replace("__", "_").replace("/", "").replace("1","one")\
                     .replace("2","two").replace("3","three").replace("4","four").replace("5","five")\
                     .replace("6","six").replace("7","seven").replace("8","eight").replace("9","nine")\
                     .replace("0","zero").replace("health_plan_or_reference_to_a_list_of_specific", "")\
                     .replace("\n","").replace("\r","").replace("\r\n","").replace("`", "")\
                     .replace("to", "too").replace(u"\u201c", "").replace("•", "")
                     for x in dataframe.columns]

     #limit column length to 64 and reading right to left
    dataframe.columns = dataframe.columns.str[-60:] 

     #adding a number if duplicated column name
    def uniquify(dataframe):
        seen = set()

        for item in dataframe:
            fudge = 1
            newitem = item

            while newitem in seen:
                fudge += 1
                newitem = "{}_{}".format(item, fudge)

            yield newitem
            seen.add(newitem)

    dataframe.columns = uniquify(dataframe)

    #replacement dictionary for sql import
    replacements = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'text',
        'datetime64': 'timestamp'
    }  

    #table schema
    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))

    #db settings and connection
    host = 'localhost'
    dbname = 'postgres'
    user = 'postgres'
    password = 'casa'

    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')

    #drop table with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created successfully'.format(tbl_name)) 
    
    #insert values to table

    #save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='cp437', errors='ignore')

    #open the csv file, save it as an object
    my_file = open(k)
    print('file opened in memory')
    
    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')
    
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    cursor.close()
    print('table {0} imported to db completed'.format(tbl_name))
    
#for loop end message
print('all tables have been successfully imported into the db')

Admission Details.csv changing to admit_details
opened database successfully
admit_details was created successfully
file opened in memory
file copied to db
table admit_details imported to db completed
Appointment.csv changing to appointment
opened database successfully
appointment was created successfully
file opened in memory
file copied to db
table appointment imported to db completed
Authorization.csv changing to authtable
opened database successfully
authtable was created successfully
file opened in memory
file copied to db
table authtable imported to db completed
Billing.csv changing to billing
opened database successfully
billing was created successfully
file opened in memory
file copied to db
table billing imported to db completed
CCF026 Multi Disciplinary Discharge.csv changing to ccf026_multidisp_dc
opened database successfully
ccf026_multidisp_dc was created successfully
file opened in memory
file copied to db
table ccf026_multidisp_dc imported to db completed
Chart Users.csv

table fl_dc_summary_v7_1 imported to db completed
FormLab Discharge Summary v7.2.csv changing to fl_dc_summary_v7_2
opened database successfully
fl_dc_summary_v7_2 was created successfully
file opened in memory
file copied to db
table fl_dc_summary_v7_2 imported to db completed
FormLab EM Progress Note v1.2.csv changing to fl_em_prog_note_v1_2
opened database successfully
fl_em_prog_note_v1_2 was created successfully
file opened in memory
file copied to db
table fl_em_prog_note_v1_2 imported to db completed
FormLab EM Progress Note v1.3.csv changing to fl_em_prog_note_v1_3
opened database successfully
fl_em_prog_note_v1_3 was created successfully
file opened in memory
file copied to db
table fl_em_prog_note_v1_3 imported to db completed
FormLab EM Progress Note v1.4.csv changing to fl_em_prog_note_v1_4
opened database successfully
fl_em_prog_note_v1_4 was created successfully
file opened in memory
file copied to db
table fl_em_prog_note_v1_4 imported to db completed
FormLab EM Progress

fl_nonadmit_v1_0 was created successfully
file opened in memory
file copied to db
table fl_nonadmit_v1_0 imported to db completed
FormLab Non-Admit V1.1.csv changing to fl_nonadmit_v1_1
opened database successfully
fl_nonadmit_v1_1 was created successfully
file opened in memory
file copied to db
table fl_nonadmit_v1_1 imported to db completed
FormLab OP BH Care Survey for Adults v1.1.csv changing to fl_op_bh_survey_v1_1
opened database successfully
fl_op_bh_survey_v1_1 was created successfully
file opened in memory
file copied to db
table fl_op_bh_survey_v1_1 imported to db completed
FormLab PA Progress Note v6.4.csv changing to fl_pa_prog_note_v6_4
opened database successfully
fl_pa_prog_note_v6_4 was created successfully
file opened in memory
file copied to db
table fl_pa_prog_note_v6_4 imported to db completed
FormLab PA Treatment Plan v5.0.csv changing to fl_pa_treatment_plan_v5_0
opened database successfully
fl_pa_treatment_plan_v5_0 was created successfully
file opened in memory


file opened in memory
file copied to db
table fl_ufs_v11_5 imported to db completed
FormLab Universal Face Sheet v11.6.csv changing to fl_ufs_v11_6
opened database successfully
fl_ufs_v11_6 was created successfully
file opened in memory
file copied to db
table fl_ufs_v11_6 imported to db completed
FormLab Universal Face Sheet v11.8.csv changing to fl_ufs_v11_8
opened database successfully
fl_ufs_v11_8 was created successfully
file opened in memory
file copied to db
table fl_ufs_v11_8 imported to db completed
Last Data Refresh.csv changing to last_data_refresh
opened database successfully
last_data_refresh was created successfully
file opened in memory
file copied to db
table last_data_refresh imported to db completed
Medication Dispense.csv changing to medication_dispense
opened database successfully
medication_dispense was created successfully
file opened in memory
file copied to db
table medication_dispense imported to db completed
Medication Schedules.csv changing to medication_sche